In [9]:
import sys
sys.path.append("..")
import json
import random
import dadagp
import os
from model_ead import TransformerXL
import make_loops as loops

In [ ]:
FILELIST_PATH = "file_list.txt"
ROOT_PATH = ""
OUTPUT_PATH = ""
NUM_SAMPLES = 25
MODEL_PATH = "../model-weights/desktop_trained/ep_30.pth.tar"

In [ ]:
with open(FILELIST_PATH, "r") as f:
    file_list =  json.load(f)
num_files = len(file_list)

In [ ]:
sampled_idxes = random.sample(range(num_files), NUM_SAMPLES)

In [ ]:
human_sample_data = {}
primers = []
for id, idx in enumerate(sampled_idxes):
    filepath = file_list[idx]
    file = os.path.join(ROOT_PATH, file_list[idx])
    with open(file, "r") as f:
        text = f.read()
    list_words = text.split("\n")

    header_data = list_words[:5]
    main_data = list_words[5:]

    num_measures = 0
    measure_idx = []
    for i,token in enumerate(main_data):
        if token == "new_measure":
            num_measures += 1
            measure_idx.append(i)
    num_segments = num_measures / 4
    measure_idx.append(len(main_data))

    rand_segment = random.randint(0, num_segments)
    human_sample_data[id] = rand_segment, filepath
    measure_start = rand_segment * 4

    start_idx = measure_idx[measure_start]
    end_idx = measure_idx[measure_start + 5]

    final_loop = header_data + main_data[start_idx:end_idx]
    primer = []
    for token in final_loop:
        if "wait:" in token:
            break
        if "artist:" in token:
            primer.append("artist:unknown_artist")
        else:
            primer.append(token)
    primers.append(primer)

    token_path = os.path.join(OUTPUT_PATH, "human", "ex_" + str(id) + ".txt")
    dadagp_path = os.path.join(OUTPUT_PATH, "human", "ex_" + str(id) + ".txt")
    file_out = open(token_path, "w")
    file_out.write("\n".join(final_loop))
    file_out.close()
    dadagp.dadagp_decode(token_path, dadagp_path)

path_json = os.path.join(OUTPUT_PATH, "sampled_loops_info.json")
with open(path_json, 'w') as f:
    json.dump(human_sample_data, f)


In [ ]:
T = 1.2
P = 0.9
NUM_BARS = 16
LOOP_SIZE = 4
MIN_LEN = 4
MIN_REP_BEATS = 2.0
for idx, primer in primers:
    generated = TransformerXL.inference_single_from_primer(MODEL_PATH, ['temperature', 'nucleus'], {'t': T ,'p': P, 'num_bars': NUM_BARS}, primer)
    #convert generated list to guitarpro
    track_list, time_signatures = loops.create_track_list(song)
    beats_per_bar = loops.get_dom_beats_per_bar(time_signatures)
    min_beats = beats_per_bar * LOOP_SIZE
    max_beats = beats_per_bar * LOOP_SIZE
    lead_mat, lead_dur, melody_seq = loops.calc_correlation(track_list, 0) #assuming first instrument is most loopable
    _, loop_endpoints = loops.get_valid_loops(melody_seq, lead_mat, lead_dur, min_len=MIN_LEN, min_beats=min_beats, max_beats=max_beats, min_rep_beats=MIN_REP_BEATS)
    #randomly pick one of the endpoints
    #convert endpoint to dadagp
    